 ## **Criação e treino do LDA**

O objetivo do LDA para este problema é construir um modelo que permita distinguir os documentos entre as várias sub-doenças através da similaridade de cada documento a cada tópico de uma subdoença. Ou seja, os documentos relativos a uma sub-doença devem ser associados apenas a um tópico.

In [1]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import scispacy
import spacy
import en_core_sci_lg
import joblib
import os
import matplotlib.pyplot as plt

from IPython.display import HTML, display

from ipywidgets import interact, Layout, HBox, VBox, Box
import ipywidgets as widgets
from IPython.display import clear_output

Em primeiro lugar vamos carregar os dados que vamos utilizar para a criação dos modelos.
Vamos ler o dataset_gastric_cancer que contém o dataset completo para um pandas dataframe e o dataset_reduced que contém as primeiras 100 palavras de cada publicação.

Também iremos ler o pubs_by_disease que contém 50 publicações de cada uma das 32 subdoenças para depois podermos usá-las para validar o modelo


In [2]:
df = pd.read_csv('data/clean.csv', sep='#')

pubs = pd.read_csv('data/pubs_by_disease.csv', sep='#')
doids=pubs["doid"].unique()

De seguida carregamos um modelo de nlp treinado para documentos de biomedicina. Assim poderemos remover as palavras com um carater, os espaços e remover as palavas com sinais de pontuação

In [3]:
# medium model
nlp=en_core_sci_lg.load(disable=["tagger", "parser", "ner"])
nlp.max_length = 3000000

def spacy_tokenizer(sentence):
    return [word for word in nlp(sentence) if not (word.is_punct or word.is_space or len(word)==1)]

Como as palavras que aparecem com grande ou pouco frequência no dataset podem ter influencia no desempenho do modelo, criamos a função create_customize_words() que irá definir se adicionamos as palavras muito frequentes e pouco frequentes às stop_words.

In [4]:
def create_customize_words():
    #read words and frequency
    words=pd.read_csv('data/word_count.csv')

    customize_stop_words=[]
    for index, row in words.iterrows():
        if(row['Frequency']>1): customize_stop_words.append(row['Word'])

    return customize_stop_words

Agora, de forma a criarmos e treinarmos o modelo, implementamos a função train_lda() que nos permite definir o treino e as suas carateristicas.
Tais como o dataset a usar e se vamos eliminar ou não as palavras mais frequentes

Fora estas pequenas alterações, o que a função faz em geral é:

1-Definir as stop_words

2-Vetorizar o texto

3-Inicializar o LDA com 32 componentes

4-Treinar o LDA com o texto vetorizado

5-Calcular as distancias de cada elemento do dataset a cada tópico

6-Executar a função print_top_words()

7-Guardas os objetos resultados para poderem ser usados mais tarde

In [5]:
def train_lda(df,folder_path,delete_by_frequency):
    if(not os.path.isdir(folder_path)):
        os.mkdir(folder_path)

    customize_stop_words=[]
    #New stop words list 
    if(delete_by_frequency): customize_stop_words = create_customize_words()

    # Mark them as stop words
    for w in customize_stop_words:
        if(not isinstance(w, float)): nlp.vocab[w].is_stop = True

    #Convert a collection of text documents to a matrix of token counts
    vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, min_df=2)

    #Learn the vocabulary dictionary and return document-term matrix.
    #The astype(‘U’) is telling numpy to convert the data to Unicode (essentially a string in python 3)
    data_vectorized = vectorizer.fit_transform(df['summary'].values.astype('U'))

    lda = LatentDirichletAllocation(n_components=32, random_state=0)
    lda.fit(data_vectorized)

    #get topic distances
    doc_topic_dist = pd.DataFrame(lda.transform(data_vectorized))

    print_top_words(lda, vectorizer, n_top_words=25)

    #joblib.dump Persist an arbitrary Python object into one file.
    joblib.dump(vectorizer, folder_path+'/vectorizer.csv')
    joblib.dump(data_vectorized,folder_path+'/data_vectorized.csv')
    joblib.dump(lda, folder_path+'/lda.csv')
    doc_topic_dist.to_csv(folder_path+'/doc_topic_dist.csv', index=False)

Também definimos uma função que irá permitir perceber o funcionamento do LDA treinado. A função print_top_words() vai apresentar na consola os tópicos textuais mais importantes para cada um dos componentes, ou seja, sub-doenças

In [6]:
def print_top_words(model, vectorizer, n_top_words):
    feature_names = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        message = "\nTopic #%d: " % topic_idx
        message += " ".join([feature_names[i]for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

De forma analisarmos as previsões do LDA para um conjunto de publicaçoes, definimos a função dna_tabs que permite para cada documento representar a similaridade do mesmo para cada um dos tópicos

In [7]:
def plot_article_dna(paper_id, width=20):
    doc_topic_dist[df["id"] == paper_id].T.plot(kind='bar', legend=None, figsize=(width, 4))
    plt.xlabel('Topic')

def dna_tabs(paper_ids):
    k = len(paper_ids)
    outs = [widgets.Output() for i in range(k)]

    tab = widgets.Tab(children = outs)
    tab_titles = ['Paper ' + str(i+1) for i in range(k)]
    for i, t in enumerate(tab_titles):
        tab.set_title(i, t)
    display(tab)

    for i, t in enumerate(tab_titles):
        with outs[i]:
            ax = plot_article_dna(paper_ids[i])
            plt.show(ax)

Para que consigamos validar o desempenho do LDA usamos o dataset pubs_by_disease. Com isto verificamos se a previsão dos documentos da mesma sub-doença pertenciam a apenas um tópico e se esse tópico não se repete para mais nenhuma subdoença

In [8]:
def pretty(doid, frequencia, indent=0):
   d=dict(enumerate(frequencia.flatten(), 0))
   print("Subdoença " + str(doid) )
   for key, value in d.items():
      if(value!=0): print('Tópico ' + str(key) + " ------ " +  str(value))
   print()

def lda_validation(doids, pubs, doc_topic_dist):
    for doid in doids:
        frequencia=np.zeros(100)
        p=pubs[pubs["doid"]==doid].id
        for id in p:
            try:
                distancias=doc_topic_dist[df["id"] == id]
                max_indice=np.argmax(distancias)
                frequencia[max_indice]+=1
            except: 
                pass
        pretty(doid, frequencia)

## Modelo 1 

Para este modelo usamos os dataset clean.csv de modo a obtermos um modelo de base

In [ ]:
train_lda(df,'modelo1',False)

In [9]:
vectorizer = joblib.load('modelo1/vectorizer.csv')
data_vectorized = joblib.load('modelo1/data_vectorized.csv')
lda = joblib.load('modelo1/lda.csv') 
doc_topic_dist = pd.read_csv('modelo1/doc_topic_dist.csv')  

C:\Users\jpmrs\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\jpmrs\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


De seguida, deixo exemplos da distribuição das publicações das subdoenças 0, 1 e 25 pelos diveros tópicos

In [10]:
#documentos da sub doença 0
papers=pubs[pubs["doid"]==doids[0]]

lista=[]
[lista.append(papers.iloc[i].id) for i in range(1,len(papers))]

dna_tabs(lista)

In [ ]:
#documentos da sub doença 0
papers=pubs[pubs["doid"]==doids[1]]

lista=[]
[lista.append(papers.iloc[i].id) for i in range(1,len(papers))]

dna_tabs(lista)

In [11]:
#documentos da sub doença 25
papers=pubs[pubs["doid"]==doids[25]]

lista=[]
[lista.append(papers.iloc[i].id) for i in range(1,len(papers))]

dna_tabs(lista)

Analisando os gráficos para estas três subdoenças podemos perceber que nem todas as publicações de uma mesma sub-doença estão associadas apenas a um tópico. No entanto podemos notar que existe uma tendencia como é o caso do tópico 15 para a doença 0, da doença 1 para o tópico 15 e da doença 25 para o tópico 3

Porém, analisando melhor a distribuição da associação das publicaçoes às sub-doenças podemos reparar através dos resultados da função seguinte que maioria dos documentos de todas as subdoenças estão a ser associados ao tópico 14.

In [12]:
lda_validation(doids, pubs, doc_topic_dist)

Subdoença 4716
Tópico 1 ------ 6.0
Tópico 3 ------ 2.0
Tópico 4 ------ 1.0
Tópico 7 ------ 2.0
Tópico 11 ------ 3.0
Tópico 14 ------ 15.0
Tópico 16 ------ 3.0
Tópico 17 ------ 3.0
Tópico 18 ------ 1.0
Tópico 25 ------ 3.0
Tópico 26 ------ 2.0
Tópico 27 ------ 6.0
Tópico 29 ------ 2.0
Tópico 30 ------ 1.0

Subdoença 8025
Tópico 1 ------ 1.0
Tópico 2 ------ 1.0
Tópico 7 ------ 3.0
Tópico 11 ------ 1.0
Tópico 14 ------ 35.0
Tópico 15 ------ 2.0
Tópico 21 ------ 1.0
Tópico 25 ------ 3.0
Tópico 27 ------ 2.0
Tópico 29 ------ 1.0

Subdoença 10538
Tópico 0 ------ 1.0
Tópico 2 ------ 1.0
Tópico 3 ------ 7.0
Tópico 5 ------ 3.0
Tópico 7 ------ 3.0
Tópico 9 ------ 5.0
Tópico 10 ------ 2.0
Tópico 12 ------ 1.0
Tópico 14 ------ 14.0
Tópico 15 ------ 4.0
Tópico 20 ------ 1.0
Tópico 21 ------ 1.0
Tópico 24 ------ 1.0
Tópico 25 ------ 4.0
Tópico 27 ------ 1.0

Subdoença 10540
Tópico 2 ------ 1.0
Tópico 3 ------ 4.0
Tópico 4 ------ 2.0
Tópico 7 ------ 22.0
Tópico 10 ------ 1.0
Tópico 14 ------ 11.0
Tó

Visto isto, fomos analisar quais era as principais componentes do tópico 14 e como podemos verificar através da função seguinte são as palavras: "gastric", "case", "tumor", "carcinoma", "stomach", "node", "adenocarcinoma", "reveal", "metastas", entre outras.

In [13]:
print_top_words(lda, vectorizer, 25)


Topic #0: risk ci cancer associ gastric studi factor ratio increas use interv confid age odd smoke adjust patient year control rr cohort subject signific hr regress

Topic #1: cancer mutat gastric gene polymorph associ genotyp studi genet famili risk allel variant ci patient analysi found result sequenc control suscept dna identifi snp case

Topic #2: patient node cancer gastric lymph metastasi tumor invas surviv resect rate factor recurr stage type earli prognosi advanc size cur egc studi clinicopatholog dissect depth

Topic #3: patient tumor year case gastrointestin gist age present diagnosi tumour symptom diseas malign clinic month gi treatment primari report tract small stromal surgeri gastric stomach

Topic #4: cancer treatment gastric clinic review diseas studi therapi use trial improv develop patient current includ recent provid research approach new advanc strategi result therapeut data

Topic #5: gastric patient gastriti cancer serum level pylori metaplasia mucosa intestin ch

In [22]:
word_count = pd.read_csv('data/word_count.csv', sep=',')
print(word_count[word_count['Word']=='gastric'])
print(word_count[word_count['Word']=='case'])
print(word_count[word_count['Word']=='tumor'])
print(word_count[word_count['Word']=='carcinoma'])
print(word_count[word_count['Word']=='stomach'])
print(word_count[word_count['Word']=='node'])
print(word_count[word_count['Word']=='adenocarcinoma'])
print(word_count[word_count['Word']=='reveal'])

      Word  Frequency
0  gastric     257759
    Word  Frequency
94  case      16335
    Word  Frequency
5  tumor      69397
         Word  Frequency
16  carcinoma      38448
       Word  Frequency
13  stomach      40408
    Word  Frequency
64  node      20697
              Word  Frequency
51  adenocarcinoma      22552
        Word  Frequency
1681  reveal       1155


Como estas palavras se encontram em grande parte das publicações, elas estão a ser associadas ao tópico 14. Posto isto é importante evitar que existam palavras muito frequentes que não permitam ao LDA ter um bom desempenho

## Modelo 2

Para este modelo usamos os dataset clean.csv adicionando as palavras em que a sua frequencia é superior a 1 às stop_words

In [ ]:
train_lda(df,'modelo2',True)

In [23]:
vectorizer = joblib.load('modelo2/vectorizer.csv')
data_vectorized = joblib.load('modelo2/data_vectorized.csv')
lda = joblib.load('modelo2/lda.csv') 
doc_topic_dist = pd.read_csv('modelo2/doc_topic_dist.csv')  

C:\Users\jpmrs\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\jpmrs\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


De seguida, deixo exemplos da distribuição das publicações das subdoenças 0, 1 e 25 pelos diveros tópicos

In [24]:
#documentos da sub doença 0
papers=pubs[pubs["doid"]==doids[0]]

lista=[]
[lista.append(papers.iloc[i].id) for i in range(1,len(papers))]

dna_tabs(lista)

In [25]:
#documentos da sub doença 1
papers=pubs[pubs["doid"]==doids[1]]

lista=[]
[lista.append(papers.iloc[i].id) for i in range(1,len(papers))]

dna_tabs(lista)

In [26]:
#documentos da sub doença 25
papers=pubs[pubs["doid"]==doids[25]]

lista=[]
[lista.append(papers.iloc[i].id) for i in range(1,len(papers))]

dna_tabs(lista)

Analisando os gráficos para estas três subdoenças podemos perceber que nem todas as publicações de uma mesma sub-doença estão associadas a um tópico. No entanto podemos notar que existe uma tendencia como é o caso do tópico 17 para a doença 0, o tópico 7 para a subdoença 1 e  o tóp
Porém, analisando melhor a distribuição da associação das publicaçoes ás sub-doenças podemos reparar através dos resultados da função seguinte que maioria dos documentos de todas as subdoenças estão a ser associados ao tópico 20.

In [27]:
lda_validation(doids, pubs, doc_topic_dist)

Subdoença 4716
Tópico 0 ------ 1.0
Tópico 1 ------ 2.0
Tópico 3 ------ 2.0
Tópico 4 ------ 3.0
Tópico 6 ------ 7.0
Tópico 7 ------ 7.0
Tópico 8 ------ 1.0
Tópico 10 ------ 1.0
Tópico 11 ------ 1.0
Tópico 17 ------ 13.0
Tópico 18 ------ 1.0
Tópico 19 ------ 1.0
Tópico 20 ------ 7.0
Tópico 27 ------ 1.0
Tópico 29 ------ 1.0
Tópico 30 ------ 1.0

Subdoença 8025
Tópico 0 ------ 5.0
Tópico 2 ------ 1.0
Tópico 6 ------ 1.0
Tópico 7 ------ 15.0
Tópico 11 ------ 6.0
Tópico 12 ------ 2.0
Tópico 17 ------ 1.0
Tópico 18 ------ 4.0
Tópico 20 ------ 13.0
Tópico 23 ------ 1.0
Tópico 27 ------ 1.0

Subdoença 10538
Tópico 0 ------ 6.0
Tópico 1 ------ 1.0
Tópico 4 ------ 1.0
Tópico 5 ------ 2.0
Tópico 7 ------ 8.0
Tópico 12 ------ 2.0
Tópico 15 ------ 1.0
Tópico 17 ------ 1.0
Tópico 18 ------ 4.0
Tópico 20 ------ 7.0
Tópico 23 ------ 1.0
Tópico 24 ------ 6.0
Tópico 27 ------ 1.0
Tópico 29 ------ 1.0
Tópico 30 ------ 3.0
Tópico 31 ------ 4.0

Subdoença 10540
Tópico 0 ------ 8.0
Tópico 5 ------ 1.0
Tópic

Este mau resultado do LDA pode ser explicado porque o LDA apenas foi treinado com palavras que apareceram uma vez e daí nao é possível concluir quais são os tópicos de cada componente

In [40]:
print_top_words(lda, vectorizer, 25)


Topic #0: biopsi diagnosi endoscop examin endoscopi diagnos histolog esophag esophagu patholog diagnost synchron tomographi metachron comput evalu ultrasonographi cytolog underw studi preoper gastrointestin histopatholog aspir varic

Topic #1: promot regul mechan studi prolifer tissu upregul involv associ downregul potenti carcinogenesi howev demonstr tumorigenesi identifi silenc indic therapeut hypermethyl epigenet correl addit furthermor underli

Topic #2: studi analysi identifi sampl associ databas includ profil potenti amplif systemat compar valu sensit evalu differenti specif analys analyz pubm heterogen prognost signatur curv embas

Topic #3: tissu correl significantli sampl compar adjac studi signific associ metastasi analysi investig immunohistochemistri prognosi analyz surviv indic differenti clinicopatholog evalu determin increas immunohistochem quantit healthi

Topic #4: incid increas femal studi observ countri decreas compar registri estim prostat significantli declin gene